<a href="https://colab.research.google.com/github/LucasBezerraSantos/Alura_Machine_Learning/blob/master/Validacao_e_Otimizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook tem como objetivo avaliar e testar técnicas de otimização de resultados.

## Bibliotecas e Pré-processamento

In [1]:
import pandas as pd
import numpy as np

In [2]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

Separação de dados

In [4]:
X = dados.drop('vendido', axis=1)
y = dados['vendido']

SEED = 14

In [5]:
dados.columns

Index(['preco', 'vendido', 'idade_do_modelo', 'km_por_ano'], dtype='object')

Instanciando o Modelo

In [6]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25, random_state= SEED, stratify= y )

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(X_train, y_train)
previsoes = modelo.predict(X_test)

acuracia = accuracy_score(y_test, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 76.60%


In [7]:
dummy = DummyClassifier()
dummy.fit(X_train, y_train)
acuracia = dummy.score(X_test, y_test) * 100

print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 58.00%


OBS. Por mais que a métrica tenha sido aparentemente positiva, o fator da aleatoriedade pode atrapalhar a precisão e aplicação dos resultados em um caso real, pois se fosse retirado o SEED do random_state, uma nova separação de treino e teste poderia retornar valores melhor ou piores do que o obtido no exemplo acima. Por esse motivo, o mais adequado é testar o modelo com diferentes separações e definir um intervalo de confiança para avaliar a métrica

## Cross-Validate

O Cross-Validation (validação cruzada) é uma técnica usada para avaliar a performance de um modelo de Machine Learning em dados não vistos. Em outras palavras, o objetivo é verificar se o modelo está aprendendo padrões gerais que podem ser aplicados a novos dados ou se está superajustando (overfitting) aos dados de treinamento.

O Cross-Validation é especialmente útil quando temos um conjunto de dados limitado, pois nos permite usar todo o conjunto de dados para treinamento e teste. Ele divide o conjunto de dados em partes menores, chamadas de folds, e em cada rodada, usa uma parte para teste e as outras para treinamento. Isso é repetido várias vezes, com cada fold sendo usado para teste em uma rodada diferente. Ao final, temos uma medida de desempenho geral do modelo, que pode ser usada para ajustar seus parâmetros e melhorar sua performance.

In [8]:
from sklearn.model_selection import cross_validate

In [9]:
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X, y, cv= 5, return_train_score= False ) # cv= número de sepação nos dados
resultados['test_score']

array([0.756 , 0.7565, 0.7625, 0.7545, 0.7595])

Esta função a seguir recebe os dados e calcula média e o intervalo de confiança com base em dois desvios padrões, e será usada com frequência no notebook

In [10]:
def imprime_resultados(resultados):
  media = resultados['test_score'].mean()
  desvio_padrao = resultados['test_score'].std()
  
  print('Média Acurácia %.2f' % (media * 100))
  print('Intervalo de Acurácia [ %.2f, %.2f ]' % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [11]:
imprime_resultados(resultados)

Média Acurácia 75.78
Intervalo de Acurácia [ 75.21, 76.35 ]


## Aleatoriedade no Cross-Validate

K-fold é uma técnica utilizada em conjunto com o cross validation para dividir um conjunto de dados em k conjuntos (ou folds) de tamanhos iguais ou aproximados. Em seguida, o modelo é treinado k vezes, em cada uma das vezes utilizando k-1 folds como conjunto de treinamento e 1 fold como conjunto de validação.

A utilidade do K-fold é permitir que todos os dados sejam usados tanto para treinamento quanto para validação, evitando assim que um subconjunto específico de dados influencie demasiadamente a qualidade do modelo final.

O cross validation, por sua vez, é uma técnica de avaliação de desempenho de modelos que consiste em dividir o conjunto de dados em conjuntos de treinamento e validação, para então avaliar o desempenho do modelo em relação aos dados de validação. O K-fold é uma variação dessa técnica, que utiliza a técnica de K-fold para gerar os conjuntos de treinamento e validação.

Dessa forma, o uso de K-fold com cross validation é especialmente útil em problemas em que os dados são limitados e se deseja obter uma estimativa mais robusta do desempenho do modelo. Ao dividir o conjunto de dados em vários conjuntos de treinamento e validação, é possível obter uma avaliação mais precisa e evitar overfitting.

### Kfold

In [12]:
from sklearn.model_selection import KFold

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X, y, cv = cv, return_train_score=False)
imprime_resultados(resultados)

Média Acurácia 75.76
Intervalo de Acurácia [ 73.85, 77.67 ]


### StratifiedKFold

O StratifiedKFold é uma variação do KFold que, ao contrário do KFold padrão que realiza a divisão dos dados em subconjuntos aleatórios, estratifica a divisão dos dados em subconjuntos de maneira que cada subconjunto possua uma distribuição similar das classes presentes nos dados originais. Em outras palavras, o StratifiedKFold preserva a proporção de amostras de cada classe presente nos dados originais em cada subconjunto criado durante a validação cruzada.

A principal diferença entre o StratifiedKFold e o KFold é que o primeiro é mais apropriado para problemas de classificação, enquanto o segundo é mais adequado para problemas de regressão ou problemas em que as classes possuem uma distribuição uniforme. Em geral, sempre que se trabalha com um conjunto de dados desbalanceado, é recomendado o uso do StratifiedKFold para validação cruzada em modelos de classificação.

In [13]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X, y, cv = cv, return_train_score=False)
imprime_resultados(resultados)

Média Acurácia 75.78
Intervalo de Acurácia [ 73.59, 77.97 ]


### Modelos de Veículos Aleatórios

Abaixo criamos grupos aleatórios para testar mais uma funcionalidade de otimização. No caso, são criados dados que representam os modelos dos veículos para que os dados sejam agrupados a partir deles.

In [15]:
np.random.seed(SEED)
dados['modelo'] = dados['idade_do_modelo'] + np.random.randint( -2, 3, size=10000 )
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1

In [20]:
dados.modelo.unique() #modelos de carros em um exemplo hipotético

array([21, 20, 16,  4,  6, 15, 18, 10, 19, 17,  3,  9, 12, 11, 23,  5, 14,
        7, 24, 13, 22,  8,  2,  1])

### GroupKFold

O GroupKFold funciona de maneira semelhante ao KFold, mas ele usa informações sobre os grupos de cada amostra para garantir que todas as amostras de um mesmo grupo estejam no mesmo conjunto. O algoritmo divide os dados em K partes, como no KFold, mas as partes são definidas de modo a ter grupos inteiros em cada parte.

Isso ajuda a evitar problemas comuns em que um modelo aprende características específicas de um grupo que não se aplicam a outros grupos, o que pode levar a um desempenho ruim em novos dados.

In [21]:
from sklearn.model_selection import GroupKFold

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, X, y, cv = cv, groups= dados.modelo, return_train_score=False) 
#groups= incluido no parâmetro para que os dados sejam separados de acordo com os grupos

imprime_resultados(resultados)

Média Acurácia 75.76
Intervalo de Acurácia [ 71.30, 80.22 ]


### Pipeline

Para testar com um novo modelo SVC, será utilizado o StandardScaler e o GroupKfold. Para otimizar o código será aplicada a função Pipeline que realizará a transformação do scaler e aplicará o modelo em cada grupo Kfold.

In [25]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

np.random.seed(SEED)
modelo = SVC()
scaler = StandardScaler()

pipeline = Pipeline([('transformação', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
resultados = cross_validate(pipeline, X, y, cv = cv, groups= dados.modelo, return_train_score=False)
imprime_resultados(resultados)

Média Acurácia 76.70
Intervalo de Acurácia [ 73.34, 80.06 ]
